In [1]:
from common.model.embedding.predefined.cbramod import CBraModFoundationEmbedder
from models.EEGAVI.base_EEGAVI.dataset import kd_train_dataset
from torch.utils.data import DataLoader
from models.EEGAVI.base_EEGAVI.EEGAVI import EEGAVI
model = EEGAVI(video_kd_size=100, audio_kd_size=100)
ds = kd_train_dataset("../../../resources/AMIGOS/processed/spec.csv")

Some weights of VivitModel were not initialized from the model checkpoint at google/vivit-b-16x2-kinetics400 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
loader = DataLoader(ds, batch_size=2)
x = next(iter(loader))

x = x[0]["EEGAVI"]
x["kd"] = True
res = model(x)

Opening raw data file ../../../resources/AMIGOS/processed/P01_31_raw.fif...
Isotrak not found
    Range : 0 ... 19885 =      0.000 ...   155.352 secs
Ready.


It is strongly recommended to pass the `sampling_rate` argument to `SeamlessM4TFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


Opening raw data file ../../../resources/AMIGOS/processed/P01_31_raw.fif...
Isotrak not found
    Range : 0 ... 19885 =      0.000 ...   155.352 secs
Ready.


It is strongly recommended to pass the `sampling_rate` argument to `SeamlessM4TFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


In [3]:
res

(tensor([[[ 1.3566e-01,  9.4241e-02,  1.1714e-01,  ...,  1.4290e-01,
            1.4221e-01, -1.2803e-01],
          [-4.1909e-02,  1.4023e-01, -9.3636e-02,  ...,  9.2475e-03,
           -3.9005e-02, -3.0738e-01],
          [-5.7567e-02, -5.8265e-02, -9.5615e-02,  ..., -1.8228e-01,
            1.2274e-01, -8.6390e-02],
          ...,
          [ 4.8325e-01,  8.9605e-03,  3.3534e-01,  ..., -6.0386e-02,
            4.5076e-01, -9.0766e-02],
          [ 3.1630e-01, -1.6252e-01,  2.3887e-01,  ..., -1.2841e-01,
            1.6497e-01, -2.6548e-02],
          [ 5.4205e-01, -1.7980e-04,  3.2390e-01,  ..., -7.3067e-04,
            3.6489e-01,  1.4896e-01]],
 
         [[-1.8636e-01,  4.5446e-01, -2.0553e-01,  ...,  2.2948e-01,
           -1.0508e-03,  8.4869e-02],
          [ 3.2817e-02,  1.6834e-01,  1.3727e-01,  ...,  6.2908e-02,
            1.4926e-01, -9.7885e-02],
          [-2.7802e-02,  2.1628e-01, -5.6152e-02,  ...,  1.3965e-01,
           -9.5949e-05, -1.8127e-01],
          ...,
    